### Code Hist.

 - CODE  
    &ensp; : Model - Transformer  

  - DATE  
    &ensp; 2024-01-06 Created  
    &emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 1)   
    
 - DESC  
    &ensp; :   

 - DATA  
  
 - Related Link  
    &ensp; : 

# 01. Code

## 01-01. Init

### 01-01-01. Init_Module Import

In [1]:
#region Basic_Import
## Basic
import os
import warnings
warnings.filterwarnings('ignore')

import numpy as np
from numpy import array, hstack

import pandas as pd
import random
from pandas import DataFrame, Series

from utils import submission_test

## Datetime
import time
import datetime as dt
from datetime import datetime, date, timedelta

import glob
from glob import glob
import requests
import json

## 시각화
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 8]

from scipy import stats

## TSNE
from sklearn.datasets import load_digits
from sklearn.manifold import TSNE

## 정규화
# from sklearn.preprocessing import MinMaxScaler, StandardScaler
## Modeling, Model Training
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
## Model 평가
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error, mean_squared_error, r2_score   # model.score   

## Excel/CSV
import openpyxl, xlrd

import urllib
from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus

from selenium import webdriver
from selenium.webdriver.chrome.service import Service

from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

## Init.
pd.options.display.float_format = '{:.10f}'.format
#endregion Basic_Import

ImportError: cannot import name 'submission_test' from 'utils' (c:\Users\Admin\anaconda3\envs\dev-ts\lib\site-packages\utils\__init__.py)

In [ ]:
## Import_DL
str_tar = "torch"
## For Torch
if str_tar == "torch":
    import torch
    import torch.nn as nn
    from torch.nn.utils import weight_norm
    print("Torch Imported")
## For TF
elif str_tar == "tf":
    import tensorflow as tf
    import tensorflow_addons as tfa
    print("Tensorflow Imported")
else:
    print("Error : Cannot be used except for Keywords")
    print(" : torch / tf")

Torch Imported


In [ ]:
## Import_Local (현재 프로그램에서 미사용)
from DEV_Common_Data_Analysis import print_desc_statistic
from DEV_Common_Data_Datetime import create_col_ymdhm, create_col_datetime, create_df_dt, conv_midnight_24to00, create_col_weekdays #, validate_date, list_invalidDate, calc_df_dt
from DEV_Common_Data_Preprocessing import resample_by_last, find_outlier_Usages, del_outlier_Usages
from DEV_Common_Data_Visualization import visualization_df
from DEV_KASI_Holiday import KASI_holiDay, KASI_restDay, KASI_anniDay
from DEV_KMA_Weather_ASOS import KMA_ASOS_DATA
from DEV_KDHC_Usage import KDHC_HEAT_Usage

### 01-01-02. Config (Directory, Params)

In [ ]:
## Init_config
SEED = 42

np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = "1"
random.seed(SEED)

In [ ]:
## Define Todate str
str_now_ymd = pd.datetime.now().date()
str_now_y = pd.datetime.now().year
str_now_m = pd.datetime.now().month
str_now_d = pd.datetime.now().day
str_now_hr = pd.datetime.now().hour
str_now_min = pd.datetime.now().minute

print(pd.datetime.now())
print(str(str_now_y) + " / " + str(str_now_m)  + " / " + str(str_now_d))
print(str(str_now_hr) + " : " + str(str_now_min))

2024-01-05 18:59:03.908913
2024 / 1 / 5
18 : 59


## 01-02. Data Load (df_raw)

### 01-02-01. KDHC Heat Usage

In [ ]:
# Define data root directory
str_dir_kdhcHeat = "./data_Energy_KDHC_National_Heat/"
# print(os.listdir(str_dir_kdhcHeat))

df_kdhc_heat = pd.read_csv(str_dir_kdhcHeat + 'KDHC_COMB_HEATxWEATHER_2019-2022.csv', index_col = 0)
df_kdhc_heat = df_kdhc_heat[['METER_DATE', 'YEAR', 'MONTH', 'DAY', 'code_day_of_the_week', 'rest_YN', 'dist_from_holiday'
                             ,'HOUR'
                            #  , 'temp_outdoor', 'temp_dew_point', 'temp_ground'
                            #  , 'humidity'
                            #  , 'rainfall'
                            #  , 'snowfall', 'snowfall_3hr'
                            #  , 'wind_speed', 'wind_direction'
                            #  , 'pressure_vapor', 'pressure_area', 'pressure_sea'
                            #  , 'visual_range'
                             , 'PAJU', 'GOYANG', 'SAMSONG', 'JOONGANG'
                             , 'KANGNAM'
                             , 'PANGYO', 'BUNDANG', 'YONGIN', 'GWANGGYO', 'SUWON', 'HWASUNG', 'DONGTAN', 'PYONGTAEK', 'CHEONGJU', 'SEJONG'
                             , 'KIMHAE', 'DAEGU', 'YANGSAN'
                             , 'GWANGJU_JEONNAM']]
## 어디서 중복 생긴건지 확인 필요
df_kdhc_heat = df_kdhc_heat.drop_duplicates()

df_kdhc_heat['METER_DATE'] = pd.to_datetime(df_kdhc_heat['METER_DATE'])
df_dt = df_kdhc_heat['METER_DATE']

## 추출된 "DATE"로부터 Week를 계산
# df_kdhc_heat['WEEK'] = df_kdhc_heat['METER_DATE'].dt.strftime('%G-%V') 

for i in range(0, len(df_kdhc_heat)):
    ## 중간에 Code화 되지 않은 rest_YN Data가 있다.
    if((str(df_kdhc_heat['rest_YN'].iloc[i]) == 'N') | (df_kdhc_heat['rest_YN'].iloc[i] == '0')):
        df_kdhc_heat['rest_YN'].iloc[i] = 0
    if((str(df_kdhc_heat['rest_YN'].iloc[i]) == 'Y') | (df_kdhc_heat['rest_YN'].iloc[i] == '1')):
        df_kdhc_heat['rest_YN'].iloc[i] = 1

    ## 중간에 이상하게 처리된 KASI Data가 있다.
    if((str(df_kdhc_heat['dist_from_holiday'].iloc[i]) == '제헌절')
        | (str(df_kdhc_heat['dist_from_holiday'].iloc[i]) == '한글날')):
        df_kdhc_heat['dist_from_holiday'].iloc[i] = 0

df_kdhc_heat['rest_YN'] = df_kdhc_heat['rest_YN'].astype('int64')
df_kdhc_heat['dist_from_holiday'] = df_kdhc_heat['rest_YN'].astype('int64')

print(df_kdhc_heat.columns)
# df_kdhc_heat

Index(['METER_DATE', 'YEAR', 'MONTH', 'DAY', 'code_day_of_the_week', 'rest_YN',
       'dist_from_holiday', 'HOUR', 'PAJU', 'GOYANG', 'SAMSONG', 'JOONGANG',
       'KANGNAM', 'PANGYO', 'BUNDANG', 'YONGIN', 'GWANGGYO', 'SUWON',
       'HWASUNG', 'DONGTAN', 'PYONGTAEK', 'CHEONGJU', 'SEJONG', 'KIMHAE',
       'DAEGU', 'YANGSAN', 'GWANGJU_JEONNAM'],
      dtype='object')


NameError: name 'nn' is not defined